In [1]:
import pandas as pd
from bs4 import BeautifulSoup 
import requests
import datetime
from datetime import date 
import time
import json
import csv
import numpy as np
import re
import lxml.html
import os
import tabula

In [2]:
wta_path = "https://wtafiles.wtatennis.com/pdf/rankings/All_YTD_Prize_Money.pdf"

In [3]:
atp_path = "https://www.protennislive.com/posting/ramr/current_prize.pdf"

In [4]:
wta_dfs = tabula.read_pdf(wta_path, stream=False, user_agent="Mozilla/5.0", pages='1-11')
# read_pdf returns list of DataFrames
print(len(wta_dfs))

11


In [5]:
print(wta_dfs[2])

    Standing                 Unnamed: 0  Name  Nat  Singles  Doubles    Mixed  \
0         93               BUREL, CLARA   NaN  FRA  479,101   20,370    5,000   
1         94              MINNEN, GREET   NaN  BEL  390,947  107,891      NaN   
2         95              NAVARRO, EMMA   NaN  USA  463,806   34,283      NaN   
3         96       PARRIZAS DIAZ, NURIA   NaN  ESP  486,764    9,125      NaN   
4         97            VOLYNETS, KATIE   NaN  USA  492,258      697      NaN   
5         98               GALFI, DALMA   NaN  HUN  459,858   26,841      NaN   
6         99           KALINSKAYA, ANNA   NaN  NaN  386,918   96,583      NaN   
7        100            MARINO, REBECCA   NaN  CAN  455,476   25,081      NaN   
8        101                JUVAN, KAJA   NaN  SLO  479,100      435      NaN   
9        102           BAINDL, KATERYNA   NaN  UKR  464,741   13,929      NaN   
10       103               KANEPI, KAIA   NaN  EST  462,316   13,374      NaN   
11       104           BRENG

In [6]:
#print(wta_dfs[2].columns)

Index(['Standing', 'Unnamed: 0', 'Name', 'Nat', 'Singles', 'Doubles', 'Mixed',
       'Total'],
      dtype='object')


In [7]:
#print(wta_dfs[10])

    Standing                           Name  Nat Singles Doubles  Mixed  \
0        461               VISMANE, DANIELA  LAT  15,390   7,094    NaN   
1        462                  LAHEY, ASHLEY  USA  15,427   6,975    NaN   
2        463               STRUPLOVA, JULIE  CZE  19,780   2,554    NaN   
3        464              COLMEGNA, MARTINA  ITA  19,059   3,159    NaN   
4        465          VAN DER HOEK, ROSALIE  NED     152  22,033    NaN   
5        466               COSTOULAS, SOFIA  BEL  17,249   4,685    NaN   
6        467                 DINU, CRISTINA  ROU  17,219   4,706    NaN   
7        468              BUYUKAKCAY, CAGLA  TUR  19,884   1,907    NaN   
8        469                  DUERST, JENNY  SUI  14,306   7,192    NaN   
9        470              KUZMOVA, KATARINA  SVK  17,959   3,407    NaN   
10       471                HEWITT, DALAYNA  USA  12,951   8,375    NaN   
11       472                 KUBKA, MARTYNA  POL  14,365   6,755    NaN   
12       473            M

In [9]:
#DO NOT USE, WILL ERROR: 
#wta_df.drop(columns=['Name'], inplace=True)

NameError: name 'wta_df' is not defined

In [ ]:
#Faulty original failed to specify that "name" was a column 
# if (len(wta_df.columns) == 7):
#     wta_df.drop('Name', inplace=True)
#     wta_df.rename(columns={'Unnamed: 0': 'Name'}, inplace=True)

In [10]:
#only keep the columns that are common to the two dfs
#wta_dfs = wta_dfs[['Name', , ]
women_df= pd.DataFrame([])

for wta_df in wta_dfs:
    if (len(wta_df.columns) == 8):
        wta_df.drop(columns=['Name'], inplace=True)
        wta_df.rename(columns={'Unnamed: 0': 'Name'}, inplace=True)
    wta_df.rename(columns={'Standing': 'Rank'}, inplace=True)   
    wta_df = wta_df[['Rank','Name', 'Total', 'Singles','Doubles' ]]
    women_df = pd.concat([women_df, wta_df])
    


In [11]:
print(women_df.columns)

Index(['Rank', 'Name', 'Total', 'Singles', 'Doubles'], dtype='object')


In [ ]:
#previously: didn't have the INPLACE parameter 

In [12]:
women_df.reset_index(drop = True, inplace=True)

In [13]:
#Cutting off at index 499
women_df = women_df[women_df['Rank'] <= 500]

In [14]:
#len(women_df)

500

In [48]:
#women_df.to_csv("files/2023-10-09_wta.csv")

In [ ]:
#some columns have missing player names - pages == resolved thanks to RENAME 

In [15]:
atp_dfs = tabula.read_pdf(atp_path, stream=True, user_agent="Mozilla/5.0", pages='1-12', pandas_options={'header': None})

In [16]:
#print(atp_dfs[0].columns)

RangeIndex(start=0, stop=6, step=1)


In [17]:
#print(atp_dfs[0])

     0                             1            2            3           4  \
0    1               Djokovic, Novak  $10,590,175  $10,581,665      $8,510   
1    2               Alcaraz, Carlos   $9,603,879   $9,603,879          $0   
2    3              Medvedev, Daniil   $7,833,244   $7,833,244          $0   
3    4                Sinner, Jannik   $5,189,328   $5,138,315     $51,013   
4    5                Rublev, Andrey   $4,731,576   $4,376,608    $354,968   
5    6           Tsitsipas, Stefanos   $4,138,173   $4,004,628    $133,545   
6    7             Zverev, Alexander   $3,645,582   $3,548,581     $97,001   
7    8                 Fritz, Taylor   $3,370,858   $3,275,210     $95,648   
8    9               Hurkacz, Hubert   $3,348,781   $3,250,532     $98,249   
9   10                  Rune, Holger   $3,157,669   $3,135,158     $22,511   
10  11                  Ruud, Casper   $3,042,290   $3,016,208     $26,082   
11  12               de Minaur, Alex   $2,693,036   $2,611,602  

In [18]:
men_df= pd.DataFrame([])

for atp_df in atp_dfs:
    atp_df.columns = ['Rank', 'Name', 'Total', 'Singles','Doubles', 'Career']
    atp_df = atp_df[['Rank','Name', 'Total', 'Singles','Doubles' ]]
    men_df = pd.concat([men_df, atp_df])

In [19]:
men_df['Total'] = men_df['Total'].str.strip('$')

In [20]:
men_df['Singles'] = men_df['Singles'].str.strip('$')

In [21]:
men_df['Doubles'] = men_df['Doubles'].str.strip('$')

In [65]:
#NOT WORKING: men_df["Rank"] = men_df["Rank"].astype(int)

In [22]:
#print(men_df.columns)

Index(['Rank', 'Name', 'Total', 'Singles', 'Doubles'], dtype='object')


In [23]:
men_df.reset_index(drop = True, inplace=True)

In [ ]:
#WE JUST LOST DJOKOVIC!!!!  --> resolved with correct tabula formula 

In [67]:
#NOT WORKING, either as int or as string 
#men_df = men_df[men_df['Rank'] <= 500]

In [24]:
#we used the index 
men_df = men_df[men_df.index <= 499]

In [25]:
#len(men_df)

500

In [19]:
#mkdir files 
#cd files 
#name each file with the date  
#also store all weekly rank data in a single file, for atp and wta ("all_weeks")

In [20]:
#already done:
#os.mkdir("files")

In [26]:
date = date.today()

In [27]:
print(date)

2023-10-22


In [28]:
women_df.to_csv(f"files/{date}_wta.csv")

In [ ]:
#PROBLEM: the index vaues repeat themselves instead of being sequential == resolved via "inplace=true
#LETS SEE AFTER WE COMBINE 

In [29]:
men_df.to_csv(f"files/{date}_atp.csv")

In [30]:
men_df['gender'] = 'male'
women_df['gender'] = 'female'

In [31]:
mixed_df = pd.concat([women_df, men_df])

In [ ]:
#print(mixed_df)

In [32]:
#print(mixed_df.iloc[55])

Rank                  56
Name       PARKS, ALYCIA
Total            762,894
Singles          533,141
Doubles          222,653
gender            female
Name: 55, dtype: object


In [33]:
#print(mixed_df.iloc[552])

Rank                    53
Name       Koolhof, Wesley
Total              989,496
Singles                  0
Doubles            989,496
gender                male
Name: 52, dtype: object


In [ ]:
#problems with women_df from last session: 
# - ranks not in sequence: 
# - no names for some pages: 
# - doubles earnings missing last 3 digits:

In [34]:
#mixed_df.reset_index(inplace=True)
mixed_df.reset_index(drop = True, inplace=True)

In [ ]:
#******** RETURN TO THIS AFTER 

In [35]:
#mixed_df.to_csv(f"files/{date}_mixed.csv")

In [81]:
#HOW/WHAT do we compare??
#compare [gender]=women at RANK i vs gender - men at rank i
#Do we keep the names or just the ranks? 
# what does DW show? 
#OPTION 1
#weekly updated table of the difference between W1 M1
#BUT if you hover over any of the rows, you get the info on absolute earnings and player name ()both are bound to change due to ranking change) 
#OPTION 2
#Show difference at each rank, plus respective names??? CTA -- hover to see actual earnings$$ 

#create a new df called "difference" loop thru mixed df by looping with gender-female from rank 1 to 500, 
# then gender-male at rank value 1 to 500, 


In [ ]:
# for player in mixed_df:
#     if mixed_df[[gender = female && index ]]

In [ ]:
# diff = 0
# for woman in women_df: 
#     difference_df = pd.DataFrame()


    
    

In [82]:
# ranks = []
# diff = []

# for index in women_df.index:
#     men_total = men_df.iloc[index]['Total']
#     women_total = women_df.iloc[index]['Total']
#     ranks.append(index + 1)
#     diff.append(men_total - women_total)
    

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [ ]:
#remove commas 

In [83]:
# men_total = int(men_df.iloc[index]['Total'].replace(',', ''))
# women_total = int(women_df.iloc[index]['Total'].replace(',', ''))

In [84]:
# ranks = []
# diff = []

# for index in women_df.index:
#     men_total = int(men_df.iloc[index]['Total'].replace(',', ''))
#     women_total = int(women_df.iloc[index]['Total'].replace(',', ''))
#     ranks.append(index + 1)
#     diff.append(men_total - women_total)

In [85]:
# len(diff)

500

In [86]:
# diff_df = pd.DataFrame([])
# diff_df['Rank'] = ranks
# diff_df['diff'] = diff

In [88]:
# diff_df.head()

,Rank,diff
0,1,3220540
1,2,3949552
2,3,2321788
3,4,396936
4,5,-190467


In [ ]:
#next: add names?? 

In [89]:
# diff_df = pd.DataFrame([])
# diff_df['Rank'] = ranks
# diff_df['diff'] = diff
# diff_df['women'] = women_df['Name']
# diff_df['men'] = men_df['Name']

In [90]:
# diff_df.head(50)

,Rank,diff,women,men
0,1,3220540,"SABALENKA, ARYNA","Djokovic, Novak"
1,2,3949552,"GAUFF, COCO","Alcaraz, Carlos"
2,3,2321788,"SWIATEK, IGA","Medvedev, Daniil"
3,4,396936,"RYBAKINA, ELENA","Sinner, Jannik"
4,5,-190467,"VONDROUSOVA, MARKETA","Rublev, Andrey"


In [ ]:
#NEXT: percentage difference 

In [36]:
#USE THE BELOW: 
ranks = []
diff = []
pct = []

for index in women_df.index:
    men_total = int(men_df.iloc[index]['Total'].replace(',', ''))
    women_total = int(women_df.iloc[index]['Total'].replace(',', ''))
    ranks.append(index + 1)
    diff.append(men_total - women_total)
    pct.append((women_total/men_total)*100)

In [37]:
diff_df = pd.DataFrame([])
diff_df['Rank'] = ranks
diff_df['diff'] = diff
diff_df['pct'] = pct
diff_df['women'] = women_df['Name']
diff_df['men'] = men_df['Name']

In [38]:
#diff_df.head(20)

,Rank,diff,pct,women,men
0,1,3035522,71.336432,"SABALENKA, ARYNA","Djokovic, Novak"
1,2,2824193,70.593205,"SWIATEK, IGA","Alcaraz, Carlos"
2,3,1856622,76.298172,"GAUFF, COCO","Medvedev, Daniil"
3,4,91891,98.229231,"RYBAKINA, ELENA","Sinner, Jannik"
4,5,410686,91.320313,"PEGULA, JESSICA","Rublev, Andrey"
5,6,-137105,103.313177,"VONDROUSOVA, MARKETA","Tsitsipas, Stefanos"
6,7,841144,76.927031,"MUCHOVA, KAROLINA","Zverev, Alexander"
7,8,572294,83.022305,"JABEUR, ONS","Fritz, Taylor"
8,9,860400,74.307069,"KVITOVA, PETRA","Hurkacz, Hubert"
9,10,750256,76.240195,"SAKKARI, MARIA","Rune, Holger"


In [39]:
#diff_df.tail(30)

,Rank,diff,pct,women,men
470,471,19130,52.714060,"HEWITT, DALAYNA","Furness, Evan"
471,472,19064,52.558232,"KUBKA, MARTYNA","Niklas-Salminen, Patrik"
472,473,19054,52.351897,"MARCINKEVICA, DIANA","Sabanov, Ivan"
473,474,19157,52.094326,"ITO, AOI","Sabanov, Matej"
474,475,19005,52.224736,"GARCIA PEREZ, GEORGINA","Mena, Facundo"
475,476,19110,51.805710,"IATCENKO, POLINA","Te, Rigele"
476,477,19144,51.459215,"JORGE, MATILDE","Harrison, Christian"
477,478,19101,51.466104,"BARTUNKOVA, NIKOLA","Bourgue, Mathias"
478,479,19020,51.391551,"BHOSALE, RUTUJA","Chandrasekar, Anirudh"
479,480,18947,51.447827,"SMITH, ALANA","Kingsley, Cannon"


In [ ]:
#SAVE THIS DIFF dataframe! --> feed to DW 

In [3]:
diff_df.to_csv(f"files/{date}_diff.csv")

In [ ]:
#ALSO SAVE TO UNDATED csv!!! 

In [4]:
diff_df.to_csv("latest_diff.csv")

In [5]:
ls 

OCT9.ipynb                files/                    pay_gap.ipynb
README.md                 old_atp_live_ranking.csv  tennisOCT3.ipynb
action.yml                old_tennispay.ipynb       tennisgpt.ipynb
atp_live_ranking.csv      old_tennisweekly.ipynb    weekly_race_to_top.csv


In [7]:
ls-la

total 6448
drwxr-xr-x  19 mfhan  staff      608 Oct 22 22:21 ./
drwxr-xr-x  10 mfhan  staff      320 Oct 15 19:37 ../
-rw-r--r--@  1 mfhan  staff    10244 Oct 22 22:20 .DS_Store
drwxr-xr-x  13 mfhan  staff      416 Oct 22 22:22 .git/
drwxr-xr-x   4 mfhan  staff      128 Oct 22 22:20 .github/
drwxr-xr-x  12 mfhan  staff      384 Oct 22 22:21 .ipynb_checkpoints/
-rw-r--r--   1 mfhan  staff    87443 Oct  9 19:05 OCT9.ipynb
-rw-r--r--   1 mfhan  staff       20 Oct 22 20:26 README.md
-rw-r--r--   1 mfhan  staff        0 Oct 22 22:21 action.yml
-rw-r--r--@  1 mfhan  staff       47 Sep 21 17:11 atp_live_ranking.csv
drwxr-xr-x  16 mfhan  staff      512 Oct 22 20:56 files/
-rw-r--r--   1 mfhan  staff    34428 Oct 22 20:04 latest_diff.csv
-rw-r--r--@  1 mfhan  staff       20 Sep 21 17:03 old_atp_live_ranking.csv
-rw-r--r--   1 mfhan  staff    11970 Oct  9 14:40 old_tennispay.ipynb
-rw-r--r--   1 mfhan  staff  1565449 Oct  3 16:06 old_tennisweekly.ipynb
-rw-r--r--   1 mfhan  staff    66950 Oct 22